In [ ]:
# Importing required libraries for the given code
import osmnx as ox
import networkx as nx
import heapq
import math

# Function definitions for the algorithms (reused from provided code)
def dijkstras_search(origin_key, goal_key, graph):
    open_queue = []
    heapq.heappush(open_queue, (0, origin_key))
    distances = {origin_key: 0.0}
    predecessors = {}

    while open_queue:
        current_distance, current_node = heapq.heappop(open_queue)
        if current_node == goal_key:
            return get_path(origin_key, goal_key, predecessors)
        for neighbor, edge_data in graph[current_node].items():
            edge_length = edge_data[0].get('length', 1)
            new_distance = current_distance + edge_length
            if neighbor not in distances or new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                predecessors[neighbor] = current_node
                heapq.heappush(open_queue, (new_distance, neighbor))
    raise ValueError("Goal not found in search.")

def distance_heuristic(state_key, goal_key, node_data):
    n1, n2 = node_data[state_key], node_data[goal_key]
    long1, lat1 = n1['x'] * math.pi / 180.0, n1['y'] * math.pi / 180.0
    long2, lat2 = n2['x'] * math.pi / 180.0, n2['y'] * math.pi / 180.0
    r = 6371000
    x1, y1, z1 = r * math.cos(lat1) * math.cos(long1), r * math.cos(lat1) * math.sin(long1), r * math.sin(lat1)
    x2, y2, z2 = r * math.cos(lat2) * math.cos(long2), r * math.cos(lat2) * math.sin(long2), r * math.sin(lat2)
    return ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2) ** 0.5

def a_star_search(origin_key, goal_key, graph):
    open_queue = []
    heapq.heappush(open_queue, (0, origin_key))
    costs = {origin_key: 0.0}
    predecessors = {}
    node_data = {node[0]: node[1] for node in graph.nodes(data=True)}

    while open_queue:
        current_f, current_node = heapq.heappop(open_queue)
        if current_node == goal_key:
            return get_path(origin_key, goal_key, predecessors)
        for neighbor, edge_data in graph[current_node].items():
            edge_length = edge_data[0].get('length', 1)
            new_cost = costs[current_node] + edge_length
            if neighbor not in costs or new_cost < costs[neighbor]:
                costs[neighbor] = new_cost
                predecessors[neighbor] = current_node
                f = new_cost + distance_heuristic(neighbor, goal_key, node_data)
                heapq.heappush(open_queue, (f, neighbor))
    raise ValueError("Goal not found in search.")

def get_path(origin_key, goal_key, predecessors):
    path = []
    current = goal_key
    while current != origin_key:
        path.append(current)
        current = predecessors[current]
    path.append(origin_key)
    path.reverse()
    return path

# Changing the nodes (different location coordinates) and rerunning
map_graph = ox.graph_from_place('Berkeley, California', network_type='drive')
new_origin = ox.distance.nearest_nodes(map_graph, X=-122.2727, Y=37.8716)  # Changed coordinates
new_destination = list(map_graph.nodes())[100]  # Selecting a different node

# Running the Dijkstra's algorithm
new_path_dijkstra = dijkstras_search(new_origin, new_destination, map_graph)

# Running the A* algorithm
new_path_a_star = a_star_search(new_origin, new_destination, map_graph)

# Plotting the new paths
fig_dijkstra, ax_dijkstra = ox.plot_graph_route(map_graph, new_path_dijkstra)
fig_astar, ax_astar = ox.plot_graph_route(map_graph, new_path_a_star)
